In [688]:
import numpy as np
import pandas as pdfrom sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [689]:
users = pd.read_json("./IUM21Z_Zad_02_01/users.jsonl", lines=True)
sessions = pd.read_json("./IUM21Z_Zad_02_01/sessions.jsonl", lines=True)
products = pd.read_json("./IUM21Z_Zad_02_01/products.jsonl", lines=True)

In [690]:
sessions['score'] = sessions['event_type'].map({'VIEW_PRODUCT':5, 'BUY_PRODUCT':5})
sessions

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,score
0,124,2021-07-30 00:57:46,102,1040,VIEW_PRODUCT,10,NaN,5
1,125,2021-05-13 04:51:17,102,1297,VIEW_PRODUCT,0,NaN,5
2,126,2021-03-26 17:23:58,102,1288,VIEW_PRODUCT,0,NaN,5
3,126,2021-03-26 17:25:28,102,1151,VIEW_PRODUCT,0,NaN,5
4,126,2021-03-26 17:26:46,102,1055,VIEW_PRODUCT,0,NaN,5
...,...,...,...,...,...,...,...,...
118386,20275,2021-06-07 01:00:33,301,1052,VIEW_PRODUCT,10,NaN,5
118387,20275,2021-06-07 01:00:44,301,1153,VIEW_PRODUCT,10,NaN,5
118388,20275,2021-06-07 01:03:47,301,1054,VIEW_PRODUCT,10,NaN,5
118389,20275,2021-06-07 01:05:02,301,1049,VIEW_PRODUCT,10,NaN,5


In [691]:
group = sessions.groupby(['user_id', 'product_id'])['score'].sum().reset_index()
group['score'] = group['score'].apply(lambda x: 5 if x>5 else x)
group = pd.pivot_table(group, values='score', index='user_id', columns='product_id')
group = group.fillna(0)
group = group.stack().reset_index()
group = group.rename(columns={0:'score'})
group['user_view'] = group['score'].apply(lambda x: 1 if x > 0 else 0)
group

,user_id,product_id,score,user_view
0,102,1001,0.0,0
1,102,1002,5.0,1
2,102,1003,5.0,1
3,102,1004,5.0,1
4,102,1005,5.0,1
...,...,...,...,...
63795,301,1315,5.0,1
63796,301,1316,5.0,1
63797,301,1317,5.0,1
63798,301,1318,5.0,1


In [692]:
std = MinMaxScaler(feature_range=(0, 1))
std.fit(group['score'].values.reshape(-1,1))
group['interaction_score'] = std.transform(group['score'].values.reshape(-1,1))

group

,user_id,product_id,score,user_view,interaction_score
0,102,1001,0.0,0,0.0
1,102,1002,5.0,1,1.0
2,102,1003,5.0,1,1.0
3,102,1004,5.0,1,1.0
4,102,1005,5.0,1,1.0
...,...,...,...,...,...
63795,301,1315,5.0,1,1.0
63796,301,1316,5.0,1,1.0
63797,301,1317,5.0,1,1.0
63798,301,1318,5.0,1,1.0


In [693]:
def price_bin(price):
    if price <= 25:
        return 0
    if price <= 50:
        return 1
    if price <= 100:
        return 2
    if price <= 250:
        return 3
    if price <= 500:
        return 4
    if price <= 1000:
        return 5
    if price <= 2000:
        return 6
    if price <= 4000:
        return 7
    else:
        return 8

In [694]:
def rating_bin(rating):
    if rating <= 0.5:
        return 0
    if rating <= 1.5:
        return 1
    if rating <= 2.5:
        return 2
    if rating <= 3.5:
        return 3
    if rating <= 4.5:
        return 4
    else:
        return 5

In [695]:
group = pd.merge(group, products, on="product_id", how="left")
group = pd.merge(group, users, on="user_id", how="left")
group = group[['user_id', 'product_id', 'product_name', 'category_path', 'price', 'user_rating', 'score', 'interaction_score', 'user_view']]
group['price'] = group['price'].apply(lambda x: price_bin(x))
group['user_rating'] = group['user_rating'].apply(lambda x: rating_bin(x))
group

,user_id,product_id,product_name,category_path,price,user_rating,score,interaction_score,user_view
0,102,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,2,5,0.0,0.0,0
1,102,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7,1,5.0,1.0,1
2,102,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,8,4,5.0,1.0,1
3,102,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,3,5.0,1.0,1
4,102,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,0,5.0,1.0,1
...,...,...,...,...,...,...,...,...,...
63795,301,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,2,0,5.0,1.0,1
63796,301,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,3,4,5.0,1.0,1
63797,301,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,6,4,5.0,1.0,1
63798,301,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,5,5,5.0,1.0,1


In [696]:
mask = np.random.rand(len(group)) < 0.8
trainset = group[mask]
testset = group[~mask]
trainset

,user_id,product_id,product_name,category_path,price,user_rating,score,interaction_score,user_view
0,102,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,2,5,0.0,0.0,0
1,102,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7,1,5.0,1.0,1
2,102,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,8,4,5.0,1.0,1
4,102,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,0,5.0,1.0,1
5,102,1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,2,2,5.0,1.0,1
...,...,...,...,...,...,...,...,...,...
63795,301,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,2,0,5.0,1.0,1
63796,301,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,3,4,5.0,1.0,1
63797,301,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,6,4,5.0,1.0,1
63798,301,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,5,5,5.0,1.0,1


In [697]:
train_matrix = pd.pivot_table(trainset, values='score', index='user_id', columns='product_id')
train_matrix = train_matrix.fillna(0)
train_matrix

product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0
103,0.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,0.0,5.0,0.0,5.0,0.0,5.0,5.0,0.0,5.0,0.0
104,0.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,...,0.0,5.0,5.0,0.0,0.0,5.0,0.0,5.0,5.0,0.0
105,5.0,0.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0,...,0.0,5.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0
106,0.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,0.0,5.0,...,5.0,5.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,5.0,5.0,5.0,0.0,5.0,5.0,5.0,0.0,0.0,5.0,...,0.0,5.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0
298,0.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,...,0.0,5.0,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0
299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [698]:
product_cat = trainset[['product_id', 'category_path', 'price', 'user_rating']].drop_duplicates('product_id')
product_cat = product_cat.sort_values(by='product_id')
product_cat

,product_id,category_path,price,user_rating
0,1001,Telefony i akcesoria;Telefony stacjonarne,2,5
1,1002,Komputery;Drukarki i skanery;Biurowe urządzeni...,7,1
2,1003,Komputery;Drukarki i skanery;Biurowe urządzeni...,8,4
322,1004,Gry i konsole;Gry na konsole;Gry Xbox 360,1,3
4,1005,Gry i konsole;Gry na konsole;Gry Xbox 360,1,0
...,...,...,...,...
314,1315,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,2,0
634,1316,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,3,4
316,1317,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,6,4
317,1318,Sprzęt RTV;Audio;Słuchawki,5,5


In [700]:
price_matrix = np.reciprocal(euclidean_distances(np.array(product_cat['price']).reshape(-1,1))+1)
euclidean_matrix1 = pd.DataFrame(price_matrix,columns=product_cat['product_id'],index=product_cat['product_id'])

rating_matrix = np.reciprocal(euclidean_distances(np.array(product_cat['user_rating']).reshape(-1,1))+1)
euclidean_matrix2 = pd.DataFrame(rating_matrix,columns=product_cat['product_id'],index=product_cat['product_id'])

tfidf_vectorizer = TfidfVectorizer()
doc_term = tfidf_vectorizer.fit_transform(list(product_cat['category_path']))
dt_matrix = pd.DataFrame(doc_term.toarray().round(3), index=[i for i in product_cat['product_id']], columns=tfidf_vectorizer.get_feature_names())
cos_similar_matrix = pd.DataFrame(cosine_similarity(dt_matrix.values),columns=product_cat['product_id'],index=product_cat['product_id'])

similarity_matrix = euclidean_matrix1.multiply(euclidean_matrix2).multiply(cos_similar_matrix)
content_matrix = train_matrix.dot(similarity_matrix)
std = MinMaxScaler(feature_range=(0, 1))
std.fit(content_matrix.values)
content_matrix = std.transform(content_matrix.values)
content_matrix = pd.DataFrame(content_matrix,columns=sorted(trainset['product_id'].unique()),index=sorted(trainset['user_id'].unique()))
content_matrix

,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
102,0.321405,0.824990,0.709047,0.882796,0.836407,0.850925,0.908127,0.909477,0.900762,0.850925,...,0.701029,0.852052,0.606706,0.655193,0.864113,0.869513,0.491161,0.746380,0.343953,0.330397
103,0.575471,0.702837,0.201644,0.786644,0.900367,0.858491,0.899215,0.903550,0.908269,0.858491,...,0.790207,0.965614,0.875697,0.943343,0.824214,1.000000,0.928247,0.534466,0.962651,0.496507
104,0.415245,0.907664,0.754612,0.689496,0.616121,0.647472,0.650982,0.661536,0.573474,0.647472,...,0.584007,0.635334,0.750963,0.644264,0.658657,0.711090,0.462631,0.694799,0.856495,0.376279
105,0.928096,0.225305,0.143027,0.656048,0.573042,0.624819,0.649872,0.682316,0.620598,0.624819,...,0.427888,0.658867,0.462222,0.671389,0.592390,0.957446,0.843700,0.948653,0.331686,0.938816
106,0.519401,0.860440,0.882139,0.818635,0.762975,0.800845,0.820579,0.846190,0.750587,0.800845,...,0.810646,0.720806,0.767256,0.768119,0.804251,0.393523,0.743061,0.764645,0.938348,0.452105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,0.926476,0.828191,0.735433,0.792816,0.759180,0.817930,0.816792,0.779403,0.810184,0.817930,...,0.571258,0.813608,0.752141,0.883885,0.746298,0.957058,0.916211,0.893098,0.413311,0.951088
298,0.510647,0.809611,0.877464,0.847676,0.840853,0.883281,0.891379,0.856364,0.835419,0.883281,...,0.905229,0.976068,0.887319,0.849069,0.854869,0.840881,0.746014,0.888772,0.456020,0.436018
299,0.081429,0.067756,0.030503,0.184524,0.161703,0.130637,0.133718,0.185860,0.149954,0.130637,...,0.095287,0.096338,0.128809,0.225781,0.162752,0.064922,0.091092,0.096651,0.116389,0.088972
300,0.009039,0.011875,0.004456,0.197631,0.153051,0.173172,0.203991,0.197068,0.175963,0.173172,...,0.063656,0.145906,0.075224,0.038505,0.156767,0.022899,0.015394,0.009331,0.022586,0.015617


In [701]:
content_df = content_matrix.stack().reset_index()
content_df = content_df.rename(columns={'level_0':'user_id','level_1':'product_id',0:'predicted_interaction'})
content_df

,user_id,product_id,predicted_interaction
0,102,1001,0.321405
1,102,1002,0.824990
2,102,1003,0.709047
3,102,1004,0.882796
4,102,1005,0.836407
...,...,...,...
63795,301,1315,0.853587
63796,301,1316,0.832476
63797,301,1317,0.822659
63798,301,1318,0.855618


In [702]:
testset = testset.merge(content_df,on=['user_id','product_id'])
testset['predicted_view'] = testset['predicted_interaction'].apply(lambda x:1 if x>=0.5 else 0)
testset

,user_id,product_id,product_name,category_path,price,user_rating,score,interaction_score,user_view,predicted_interaction,predicted_view
0,102,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,3,5.0,1.0,1,0.882796,1
1,102,1011,BioShock Infinite (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,3,0,5.0,1.0,1,0.763627,1
2,102,1023,Mass Effect 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,2,4,5.0,1.0,1,0.837430,1
3,102,1024,Mortal Kombat (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,2,3,0.0,0.0,0,0.822557,1
4,102,1038,LCD Asus VK278Q,Komputery;Monitory;Monitory LCD,6,2,0.0,0.0,0,0.715626,1
...,...,...,...,...,...,...,...,...,...,...,...
12790,301,1294,Sencor SDA-500,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,3,3,0.0,0.0,0,0.699099,1
12791,301,1296,Sencor SDA-630,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,2,5,0.0,0.0,0,0.575341,1
12792,301,1303,Wiwa AN 220,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,2,1,5.0,1.0,1,0.450035,0
12793,301,1308,One For All SV 9143,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,2,4,5.0,1.0,1,0.669912,1


In [703]:
correct = 0
for index, record in testset.iterrows():
    if record['user_view'] == record['predicted_view']:
        correct = correct + 1
correct = correct / len(testset)
print(f"Accuracy: {correct*100}%")

Accuracy: 61.21141070730754%
